This project will take you through the process of mashing up data from two different APIs to make movie recommendations. The TasteDive API lets you provide a movie (or bands, TV shows, etc.) as a query input, and returns a set of related items. The OMDB API lets you provide a movie title as a query input and get back data about the movie, including scores from various review sites (Rotten Tomatoes, IMDB, etc.).

You will put those two together. You will use TasteDive to get related movies for a whole list of titles. You’ll combine the resulting lists of related movies, and sort them according to their Rotten Tomatoes scores (which will require making API calls to the OMDB API.)

To avoid problems with rate limits and site accessibility, we have provided a cache file with results for all the queries you need to make to both OMDB and TasteDive. Just use requests_with_caching.get() rather than requests.get(). If you’re having trouble, you may not be formatting your queries properly, or you may not be asking for data that exists in our cache. We will try to provide as much information as we can to help guide you to form queries for which data exists in the cache.

Your first task will be to fetch data from TasteDive. The documentation for the API is at https://tastedive.com/read/api.

Define a function, called get_movies_from_tastedive. It should take one input parameter, a string that is the name of a movie or music artist. The function should return the 5 TasteDive results that are associated with that string; be sure to only get movies, not other kinds of media. It will be a python dictionary with just one key, ‘Similar’.

Try invoking your function with the input “Black Panther”.

HINT: Be sure to include only q, type, and limit as parameters in order to extract data from the cache. If any other parameters are included, then the function will not be able to recognize the data that you’re attempting to pull from the cache. Remember, you will not need an api key in order to complete the project, because all data will be found in the cache.

The cache includes data for the following queries:

![alt text](image.png)

In [1]:
import json
import requests


# some invocations that we use in the automated tests; uncomment these if you are getting errors and want better error messages
# get_movies_from_tastedive("Bridesmaids")
# get_movies_from_tastedive("Black Panther")

def get_movies_from_tastedive(search):
    url = "https://tastedive.com/api/similar"
    param = {}
    param['q'] = search
    param['type'] = 'movie'
    param['limit'] = 5
    param['k'] = 'Your API key here'
    result = requests.get(url, params=param)
    return json.loads(result.text)


#get_movies_from_tastedive('Black Panther')
get_movies_from_tastedive("Tony Bennett")

{'similar': {'info': [{'name': 'Tony Bennett', 'type': 'music'},
   {'name': 'Tony Bennett', 'type': 'person'},
   {'name': 'Tony  Bennett', 'type': 'person'}],
  'results': [{'name': 'Bombshell', 'type': 'movie'},
   {'name': 'Rocketman', 'type': 'movie'},
   {'name': 'A Beautiful Day in the Neighborhood', 'type': 'movie'},
   {'name': 'Judy', 'type': 'movie'},
   {'name': 'On the Basis of Sex', 'type': 'movie'}]}}

Please copy the completed function from above into this active code window. Next, you will need to write a function that extracts just the list of movie titles from a dictionary returned by get_movies_from_tastedive. Call it extract_movie_titles.

In [2]:
def get_movies_from_tastedive(search):
    url = "https://tastedive.com/api/similar"
    param = {}
    param['q'] = search
    param['type'] = 'movie'
    param['limit'] = 5
    param['k'] = 'Your API key here'
    result = requests.get(url, params=param)
    return json.loads(result.text)


# some invocations that we use in the automated tests; uncomment these if you are getting errors and want better error messages
# extract_movie_titles(get_movies_from_tastedive("Tony Bennett"))
# extract_movie_titles(get_movies_from_tastedive("Black Panther"))

def extract_movie_titles(dicts):
    movie_titles = []
    for result in dicts['similar']['results']:
        movie_titles.append(result['name'])
    return movie_titles

#extract_movie_titles(get_movies_from_tastedive("Black Panther"))
extract_movie_titles(get_movies_from_tastedive("Tony Bennett"))

['Bombshell',
 'Rocketman',
 'A Beautiful Day in the Neighborhood',
 'Judy',
 'On the Basis of Sex']

Please copy the completed functions from the two code windows above into this active code window. Next, you’ll write a function, called get_related_titles. It takes a list of movie titles as input. It gets five related movies for each from TasteDive, extracts the titles for all of them, and combines them all into a single list. Don’t include the same movie twice.

In [3]:
def get_movies_from_tastedive(search):
    url = "https://tastedive.com/api/similar"
    param = {}
    param['q'] = search
    param['type'] = 'movie'
    param['limit'] = 5
    param['k'] = 'Your API key here'
    result = requests.get(url, params=param)
    return json.loads(result.text)

def extract_movie_titles(dicts):
    movie_titles = []
    for result in dicts['similar']['results']:
        movie_titles.append(result['name'])
    return movie_titles


# some invocations that we use in the automated tests; uncomment these if you are getting errors and want better error messages
# get_related_titles(["Black Panther", "Captain Marvel"])
# get_related_titles([])

def get_related_titles(title_list):
    related_titles = []
    for title in title_list:
        related_titles.extend(extract_movie_titles(get_movies_from_tastedive(title))) #Extend makes it so it's one huge list instead of one list per movie title
    return list(set(related_titles)) #Set takes care of duplicates (its one of the data types built in python) and we convert it back to a list because is the type that the program is expecting

get_related_titles(["Black Panther", "Captain Marvel"])
#get_related_titles([])

['Knives Out',
 'Avengers: Endgame',
 'AK vs AK',
 'Spider-Man: Far from Home',
 'Venom',
 'Ant-Man and the Wasp',
 'Avengers: Infinity War',
 'Dil Bechara',
 'Uri: The Surgical Strike',
 'Mardaani 2']

Your next task will be to fetch data from OMDB. The documentation for the API is at https://www.omdbapi.com/

Define a function called get_movie_data. It takes in one parameter which is a string that should represent the title of a movie you want to search. The function should return a dictionary with information about that movie.

For the queries on movies that are already in the cache, you won’t need an api key. You will need to provide the following keys: t and r. As with the TasteDive cache, be sure to only include those two parameters in order to extract existing data from the cache.

In [4]:

# some invocations that we use in the automated tests; uncomment these if you are getting errors and want better error messages
# get_movie_data("Venom")
# get_movie_data("Baby Mama")

def get_movie_data(movie_title):
    url = 'http://www.omdbapi.com/?apikey=[YourAPIKey]'
    # The documentation will put it like [YourAPIKey]& but the & will be added when you do requests.get() so don't put it here or else it won't work, it will give you a response 401, 
    # which means unauthorized, so basically the API says that your key is invalid, also check the email for the API Key and make sure you activate it.
    param = {}
    param['t'] = movie_title
    param['r'] = 'json'
    results = requests.get(url, params=param)
    return json.loads(results.text)

get_movie_data('Deadpool & Wolverine')

{'Title': 'Deadpool & Wolverine',
 'Year': '2024',
 'Rated': 'R',
 'Released': '26 Jul 2024',
 'Runtime': '128 min',
 'Genre': 'Action, Adventure, Comedy',
 'Director': 'Shawn Levy',
 'Writer': 'Ryan Reynolds, Rhett Reese, Paul Wernick',
 'Actors': 'Ryan Reynolds, Hugh Jackman, Emma Corrin',
 'Plot': 'Deadpool is offered a place in the Marvel Cinematic Universe by the Time Variance Authority, but instead recruits a variant of Wolverine to save his universe from extinction.',
 'Language': 'English, French, Spanish',
 'Country': 'United States, United Kingdom, Australia, New Zealand, Canada',
 'Awards': '4 wins & 1 nomination',
 'Poster': 'https://m.media-amazon.com/images/M/MV5BZmQxZWM5MzgtY2EzZC00OGUxLWE0Y2EtMDIwOTFlNmQ5MWMyXkEyXkFqcGc@._V1_SX300.jpg',
 'Ratings': [{'Source': 'Internet Movie Database', 'Value': '8.0/10'},
  {'Source': 'Rotten Tomatoes', 'Value': '78%'}],
 'Metascore': 'N/A',
 'imdbRating': '8.0',
 'imdbVotes': '266,068',
 'imdbID': 'tt6263850',
 'Type': 'movie',
 'DVD'

Please copy the completed function from above into this active code window. Now write a function called get_movie_rating. It takes an OMDB dictionary result for one movie and extracts the Rotten Tomatoes rating as an integer. For example, if given the OMDB dictionary for “Black Panther”, it would return 97. If there is no Rotten Tomatoes rating, return 0.

In [5]:
def get_movie_data(movie_title):
    url = 'http://www.omdbapi.com/?apikey=[YourAPIKey]'
    # The documentation will put it like [YourAPIKey]& but the & will be added when you do requests.get() so don't put it here or else it won't work, it will give you a response 401, 
    # which means unauthorized, so basically the API says that your key is invalid, also check the email for the API Key and make sure you activate it.
    param = {}
    param['t'] = movie_title
    param['r'] = 'json'
    results = requests.get(url, params=param)
    return json.loads(results.text)


# some invocations that we use in the automated tests; uncomment these if you are getting errors and want better error messages
# get_movie_rating(get_movie_data("Deadpool 2"))

def get_movie_rating(movie_dict):
    for critic in movie_dict['Ratings']:
        if critic['Source'] == 'Rotten Tomatoes':
            return int(critic['Value'][:-1]) #We use slicing here to get rid of the percentage which is at the end of the string, and we convert it into an int as the instructions say.
    return 0

get_movie_rating(get_movie_data('Black Panther'))

96

Now, you’ll put it all together. Don’t forget to copy all of the functions that you have previously defined into this code window. Define a function get_sorted_recommendations. It takes a list of movie titles as an input. It returns a sorted list of related movie titles as output, up to five related movies for each input movie title. The movies should be sorted in descending order by their Rotten Tomatoes rating, as returned by the get_movie_rating function. Break ties in reverse alphabetic order, so that ‘Yahşi Batı’ comes before ‘Eyyvah Eyvah’.

In [14]:
def get_movies_from_tastedive(search):
    url = "https://tastedive.com/api/similar"
    param = {}
    param['q'] = search
    param['type'] = 'movie'
    param['limit'] = 5
    param['k'] = 'Your API key here'
    result = requests.get(url, params=param)
    return json.loads(result.text)

def extract_movie_titles(dicts):
    movie_titles = []
    for result in dicts['similar']['results']:
        movie_titles.append(result['name'])
    return movie_titles

def get_related_titles(title_list):
    related_titles = []
    for title in title_list:
        related_titles.extend(extract_movie_titles(get_movies_from_tastedive(title))) #Extend makes it so it's one huge list instead of one list per movie title
    return list(set(related_titles)) #Set takes care of duplicates (its one of the data types built in python) and we convert it back to a list because is the type that the program is expecting


def get_movie_data(movie_title):
    url = 'http://www.omdbapi.com/?apikey=[YourAPIKey]'
    #The documentation will put it like [YourAPIKey]& but the & will be added when you do requests.get() so don't put it here or else it won't work, it will give you a response 401, 
    # which means unauthorized, so basically the API says that your key is invalid, also check the email for the API Key and make sure you activate it.
    param = {}
    param['t'] = movie_title
    param['r'] = 'json'
    results = requests.get(url, params=param)
    return json.loads(results.text)

def get_movie_rating(movie_dict):
    for critic in movie_dict['Ratings']:
        if critic['Source'] == 'Rotten Tomatoes':
            return int(critic['Value'][:-1]) #We use slicing here to get rid of the percentage which is at the end of the string, and we convert it into an int as the instructions say.
    return 0

# some invocations that we use in the automated tests; uncomment these if you are getting errors and want better error messages
# get_sorted_recommendations(["Bridesmaids", "Sherlock Holmes"])

def get_sorted_recommendations(movie_list):
    recommendations = {}
    related_titles = get_related_titles(movie_list)
    for title in related_titles:
        recommendations[title] = get_movie_rating(get_movie_data(title))

    return [i[0] for i in sorted(recommendations.items(), key= lambda item: (item[0], item[1]), reverse=True)] #We return a list with the names of the movies, we use list comprehension and we iterate through a sor
    # Here we return a list of the movie titles that are recommended in a sorted way
    # We use sorted() (this gives us back a list of sorted list of the specified iterable object) to iterate through the items of the dictionary recommendations, 
    # recommendations.items() gives us a list of tuples, each tuple contains the title and the rating score respectively
    # the key is the same tuple, we do this so the sorted() function first orders it by movie title and then by movie score, 
    # and finally we put reverse=True so that it's ordered from Z to A.


#get_sorted_recommendations(['Black Panther'])
#get_sorted_recommendations(["Bridesmaids", "Sherlock Holmes"])
get_sorted_recommendations(['Thelma', 'Good One', 'Wolfs', 'Late Night With The Devil', 'Insidious'])

['Till',
 'Thoroughbreds',
 'The King of Staten Island',
 'The Hunt',
 'The Farewell',
 'The Conjuring',
 'The Art of Self-Defense',
 'Sinister',
 'Promising Young Woman',
 'Priscilla',
 'Palm Springs',
 'Official Competition',
 'Late Night',
 'Kajillionaire',
 'Irresistible',
 'Insidious: Chapter 3',
 'Insidious: Chapter 2',
 'I Care a Lot',
 'Emergency',
 'Dead Silence',
 'Brittany Runs a Marathon',
 'Bombshell',
 'A Beautiful Day in the Neighborhood']